In [ ]:
!pip install folium==0.2.1
!pip install cdlib
!pip install leidenalg
!pip install imgaug==0.2.5
!pip install python-igraph

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=712cf5bd9e5a63021aec18aafed8e8e44a06d1dc2a0df1e9d2f876b95bef53a1
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 122kB 2.8MB/s 
     |████████████████████████████████| 6.4MB 75kB/s 
     |████████████████████████████████| 1.9MB 45.3MB/s 
     |████████████████████████████████| 13.6MB 306kB/s 
     |████████████████████████████████| 1.8MB 45.1MB/s 
     |████████████████████████████████| 1.8MB 44.5MB/s 
  Created wheel for karateclub: filename=karateclub-1.0.9-cp36-none-any.whl size=69929 sha256=692d2685cadf646d47f93628051ff8aaf908674cbefe8e7af76f54530ee81be7
  Stored in directory: /root/.cache/pip/wheels/06/6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pickle
import math
import igraph
import pandas as pd
import numpy as np
from igraph import *
import matplotlib.pyplot as plt
import time
import itertools
import random
from random import sample

In [ ]:
class weightedCommunity:
    # Strength of a node
    def strength(self, node):
        strength = 0
        for j in self.G.neighbors(node):
            strength += self.G.es[self.G.get_eid(node, j)]['weight']
        return strength

    # Belonging degree
    def belonging_degree(self, node, community):
        strength = 0
        strength = sum([self.G.es[self.G.get_eid(node, i)]['weight'] for i in self.G.neighbors(node) if i in community])
        return strength/self.strength(node)

    # Modularity of a graph
    def modularity(self):
        Q_new = 0
        for c in self.communities:
            pair_components = list(itertools.combinations(c, 2))
            pair_components = sample(pair_components, 1000) # Reducing execution time!
            for pair_c in pair_components:
                e_id = self.G.get_eid(pair_c[0], pair_c[1], error = False)
                weight = self.G.es[e_id]['weight'] if e_id != -1 else 0
                Q_new += self.belonging_degree(pair_c[0], c)*self.belonging_degree(pair_c[1], c)*(weight-self.strength(pair_c[0])*self.strength(pair_c[1])/(2*self.L))
        
        Q_new = Q_new/(2*self.L)
            
        return Q_new
  
    def allStrengths(self):
        for node in self.G.vs:
            self.strengths.append(self.strength(node))
    
        # Updating graph
        self.G.vs['strength'] = self.strengths

    def strongestNotLabeled(self):
        # Getting indices labeled with 'F'
        indices = [i for i in range(len(self.G.vs)) if self.G.vs[i]['label']=='F']
        
        # Getting strengths for such items
        ss = {}
        for i in indices:
            ss[self.strengths[i]] = i
        
        # Returning (one of) the strongest node index(es)
        return ss[max(ss.keys())]

    def nodesRemotion(self, c, min_bel_degree):
        old_c = 0
        while(old_c != len(c)):
            old_c = 0
            c_list = [el for el in c]
            for node in c_list:
                old_c = len(c_list)
                if(self.belonging_degree(node, c) < min_bel_degree):
                    try:
                        c.remove(node)
                    except:
                        pass
        return c

    def initialCommunityDetection(self):
        c = set()
        self.communities.append(c)
        # Getting strongest labeled with 'F'
        strongest = self.strongestNotLabeled()
        
        # Adding strongest to community
        c.add(strongest)
        self.G.vs[strongest]['label'] = 'T'

        print("Stronger added. Adding neighbors...")
        # Adding strongest's neighbors
        for neighbor in self.G.neighbors(strongest):
            c.add(neighbor)

        # Until convergence
        # if belonging degree of the community nodes is lower than 0.5, we remove them.
        print("Removing nodes < min_bel_degree...")
        c = self.nodesRemotion(c, self.min_bel_degree)

        return c

    def find_initial_community_neighbors(self, c):
        ### Finding initial community neighbors
        c_neighbors = set()

        for node in c:
            for nb in self.G.neighbors(node):
                c_neighbors.add(nb)
        return c_neighbors

    def define_nu_nlu(self, c, c_neighbors):
        """### Find Nu (b > 0.5) and Nlu (0.4 < b < 0.5) starting from the set of neighbors"""
        nu = set()
        nlu = set()

        for nb in c_neighbors:
          # Nu set: neighbor with belonging degree >= min_bel_degree
          if(self.belonging_degree(nb, c) >= self.min_bel_degree):
            nu.add(nb)
          # Nlu set neighbor with threshold < belonging degree < min_bel_degree
          elif(self.belonging_degree(nb, c) > self.threshold_bel_degree): # to be parametrized
            nlu.add(nb)
            
        return nu, nlu

    def add_nlu_to_community(self, c, nlu):
        while len(nlu) > 0:
            print("NLU size: " + str(len(nlu)))
            Q = self.modularity() # Necessario calcolarlo qua perché potrebbero 
                                        # esserci state modifiche alla community nei 
                                        # passaggi precedenti
            candidate = nlu.pop()
            c.add(candidate)
            Q_new = self.modularity() #INSERT LIST OF COMMUNITES
            if (Q_new > Q):
                Q = Q_new
            else:
                c.remove(candidate)
        return c

    def expandCommunity(self, c):
        """##  Expanding the community
        """
        c_old = 0
        nu, nlu = set(), set()
        
        while(c_old != len(c)): # While it adds nodes belonging to nlu in the community
            print("nlu_size: " + str(len(nlu)))
            while(c_old != len(c)): # While it adds nodes belonging to nu in the community
                c_neighbors = self.find_initial_community_neighbors(c)
                print("Initial community neighbors found")
                
                # Computing nu and nlu
                nu, nlu = self.define_nu_nlu(c, c_neighbors)

                # Add Nu nodes to initial community and return to neighbors search
                c_old = len(c)
                c = c.union(nu)

                # Asserting size
                if(len(c) != len(c) + len(nu) - len(c.intersection(nu))):
                    print('Incompatible sizes')
                    return -1
            
            
            # Add the nodes of Nlu to the community only if their presence increases the community modularity Q0
            c_old = len(c)
            c = self.add_nlu_to_community(c, nlu)

        # Mark the community nodes with 'T' label
        self.G.vs['label'] = [v['label'] if not v.index in c else 'T' for v in self.G.vs]
        
        # Updating T label counter
        self.T = len([v for v in self.G.vs['label'] if v == 'T'])
        
        return c

    def getCommunities(self):
        return self.communities
    
    def __init__(self, G, min_bel_degree, threshold_bel_degree):
        self.G = G
        self.N = len(self.G.vs)
        self.L = len(self.G.es)
        self.min_bel_degree = min_bel_degree
        self.threshold_bel_degree = threshold_bel_degree
        self.communities = []
        self.strengths = []
        self.T = 0
        
        # Labels
        self.G.vs['label'] = ['F']*self.N
    
    def computeCommunity(self):
        if(len(self.communities) == 0): # Just the 1st time
            # Strenghts
            self.allStrengths()
            print("Strengths added")

        # Initial community detection
        c = self.initialCommunityDetection()
        print("Initial community detected. Expanding...")

        # Community expansion
        c = self.expandCommunity(c)
        print("Community expanded")
        print("Covered nodes: " + str(self.T))
        
    def computeCommunities(self):
        while(self.N != self.T):
            self.computeCommunity()
        return self.getCommunities()
            


In [ ]:
def load_obj(name):
    with open('/content/drive/Shared drives/SNA/NetworkScience/AcademicGraph/data-collection-2020_sabiu/data/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open('/content/drive/Shared drives/SNA/NetworkScience/AcademicGraph/data-collection-2020_sabiu/data/obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
g = load_obj("graph")
components = g.clusters(mode="weak")
biggerSubG = components.subgraph(22) # 22 = Bigger connected component index in components list
g = biggerSubG

In [ ]:
comm = weightedCommunity(g, 0.50, 0.50)

In [ ]:
comm.computeCommunities()

In [ ]:
comms = comm.getCommunities()

In [ ]:
# Stop

In [ ]:
sum([len(com) for com in comms])

29782

In [ ]:
g.communities = []
for com in comms:
  g.communities.append(com)


In [ ]:
from cdlib import evaluation
from cdlib.utils import convert_graph_formats
import networkx as nx

In [ ]:
nxG = convert_graph_formats(biggerSubG, nx.Graph, directed=None)

intEdgDensity = evaluation.internal_edge_density(nxG, g).score
avgNodeDegree = evaluation.average_internal_degree(nxG, g).score
modularity = evaluation.modularity_density(nxG, g).score
conductance = evaluation.conductance(nxG, g).score

In [ ]:
intEdgDensity

0.05744160966112636

In [ ]:
avgNodeDegree

2.2267896901604707

In [ ]:
modularity

832.0744601741545

In [ ]:
conductance

0.06497096584233007

### Comparison

In [ ]:
from cdlib import algorithms

In [ ]:
kCli = algorithms.kclique(biggerSubG, 3)

In [ ]:
louv = algorithms.louvain(biggerSubG)

In [ ]:
lab = algorithms.label_propagation(biggerSubG)

In [ ]:
dem = load_obj("demon")

# Evaluation

In [ ]:
labels = ['Louvains', 'k-Clique', 'Demon', 'Custom']

In [ ]:
coms = [louv, kCli, dem, g]

d_evaluation = {}

for i, com in enumerate(coms):
  d_evaluation[i] = evaluation.f1(lab, com)
  

NameError: ignored

In [ ]:
df = pd.DataFrame(d_evaluation)
df.columns = ['Louvains', 'k-Clique', 'Demon', 'Custom']

print(df)

   Louvains  k-Clique     Demon    Custom
0   0.00024   0.49206  0.352214  0.113436
1       NaN       NaN       NaN       NaN


In [ ]:
d_evaluation_omega = {}

for i, com in enumerate(coms):
  d_evaluation_omega[i] = evaluation.omega(lab, com)

In [ ]:
df = pd.DataFrame(d_evaluation_omega)
df.columns = ['Louvains', 'k-Clique', 'Demon', 'Custom']

print(df)